In [92]:
import pandas as pd
import yaml
import chardet
import ujson
import numpy as np

In [3]:
def tax_placement(pident):
    if pident >= tax_cutoffs['species']:
        out = 'species'
    elif pident >= tax_cutoffs['genus']:
        out = 'genus'
    elif pident >= tax_cutoffs['family']:
        out = 'family'
    elif pident >= tax_cutoffs['order']:
        out = 'order'
    elif pident < tax_cutoffs['order']:
        out = 'class'
    return out



In [4]:
#Load Taxonomy File

tax_file = '/vortexfs1/omics/alexander/data/mmetsp/reference_dir/taxonomy-table.txt'
with open(tax_file, 'rb') as f:
    result = chardet.detect(f.read())  # or readline if the file is large
tax_table = pd.read_csv(tax_file, sep='\t',encoding=result['encoding'])
tax_table.columns = tax_table.columns.str.lower()
tax_table=tax_table.set_index('source_id')
#Read in tax-cutoffs
with open("tax-cutoffs.yaml", 'r') as stream:
    tax_cutoffs = yaml.safe_load(stream)


In [5]:
tax_table

,cyverse_path,source_id_dup,ref_status,strain,supergroup,division,class,order,family,genus,species,notes
source_id,,,,,,,,,,,,
MMETSP0290,2225.0,MMETSP0290,Clean,CCMP2878,Alveolata,Apicomplexa,Colpodellidea,Colpodellida,Chromeraceae,Chromera,Chromera velia,Curated by J. del Campo (PR2)
MMETSP0288,2224.0,MMETSP0288,Clean,CCMP3155,Alveolata,Apicomplexa,Colpodellidea,Vitrelladida,Vitrellaceae,Vitrella,Vitrella brassicaformis,Adl et al. 2019; Curated by J. del Campo and V...
MMETSP1451,2282.0,MMETSP1451,Clean,CCMP3346,Alveolata,Apicomplexa,Colpodellidea,Vitrelladida,Vitrellaceae,Vitrella,Vitrella brassicaformis,Adl et al. 2019; Curated by J. del Campo and V...
MMETSP0372,1822.0,MMETSP0372,Clean,Grappler Inlet BC,Alveolata,Apicomplexa,Gregarinomorphea,Eugregarinorida,Lecudinidae,Lankesteria,Lankesteria abbottii,Adl et al. 2019; Curated by J. del Campo (PR2)
MMETSP0125,1740.0,MMETSP0125,Clean,ATCC 50986,Alveolata,Ciliophora,Colpodea,Colpodea_X,Cyrtolophosidida,Aristerostoma,Aristerostoma sp.,Ciliophora EukRef curation Boscaro V.; previou...
...,...,...,...,...,...,...,...,...,...,...,...,...
MMETSP0990,1938.0,MMETSP0990,Clean,CCMP2098,Stramenopiles,Ochrophyta,Dictyochophyceae,Dictyochophyceae_X,Dictyochophyceae_XX,Uncertain,Uncertain,Not described; listed as Dictyochophyceae
MMETSP0991,1943.0,MMETSP0991,Clean,CCMP2098,Stramenopiles,Ochrophyta,Dictyochophyceae,Dictyochophyceae_X,Dictyochophyceae_XX,Uncertain,Uncertain,Not described; listed as Dictyochophyceae
MMETSP0992,1944.0,MMETSP0992,Clean,CCMP2098,Stramenopiles,Ochrophyta,Dictyochophyceae,Dictyochophyceae_X,Dictyochophyceae_XX,Uncertain,Uncertain,Not described; listed as Dictyochophyceae


In [6]:
%%timeit
prot_file = '/vortexfs1/omics/alexander/data/mmetsp/reference_dir/protein-table.txt'
pdict = {}
with open(prot_file, 'r')as f:
    for line in f:
        protein, mmetsp = line.split('\t')
        pdict[protein]=mmetsp.strip()


FileNotFoundError: [Errno 2] No such file or directory: '/vortexfs1/omics/alexander/data/mmetsp/reference_dir/protein-table.txt'

In [120]:
output_table = 'output/MAGs/diamond/IO-all-DCM-0-8-5-00_bin-230.diamond.out'
df = pd.read_csv(output_table, sep = '\t', header = None)
df.columns = ['qseqid', 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore']
df['ssqidMMETSP']=df.sseqid.map(imp)

In [126]:
# For MetaTranscriptomes
classification_df = pd.DataFrame(columns = ['classification_level', 'classification', 'max_pid'])
for t,dd in df.groupby('qseqid'):
    md = dd.pident.max()
    ds = list(set(dd[dd.pident==md]['ssqidMMETSP']))
    assignment = tax_placement(md)
    if len(ds)==1:
        classification = tax_table.loc[ds[0], assignment]
    else:
        classification_0 = []
        for d in ds:
            classification_0.append(tax_table.loc[ds[0], assignment])
            if len(set(classification_0)) ==1:
                classification = list(set(classification_0))[0]
            else:
                pass
    classification_df.loc[t] = [assignment, classification, md]
   

In [127]:
# For MetaGenomes
level_dict = {'class':['supergroup','division','class'],
                       'order':['supergroup','division','class', 'order'],
                       'family':['supergroup','division','class', 'order', 'family'],
                       'genus': ['supergroup','division','class', 'order', 'family','genus'],
                        'species':['supergroup','division','class', 'order', 'family','genus', 'species']}
classification_df = pd.DataFrame(columns = ['classification_level', 'full_classification','classification', 'max_pid'])
for t,dd in df.groupby('qseqid'):
    md = dd.pident.max()
    ds = list(set(dd[dd.pident==md]['ssqidMMETSP']))
    assignment = tax_placement(md)
    full_assignment = level_dict[assignment]
    if len(ds)==1:
        classification = tax_table.loc[ds[0], assignment]
        full_classification = tax_table.loc[ds[0], full_assignment]
        full_classification='; '.join(list(full_classification))
    else:
        classification_0 = []
        full_classification_0=[]
        for d in ds:
            classification_0.append(tax_table.loc[ds[0], assignment])
            fc = tax_table.loc[ds[0], full_assignment]
            full_classification_0.append('; '.join(list(fc)))
            if len(set(classification_0)) ==1:
                classification = list(set(classification_0))[0]
                full_classification=list(set(full_classification_0))[0]
            else:
                pass
    classification_df.loc[t] = [assignment, full_classification, classification, md]
classification_df

,classification_level,full_classification,classification,max_pid
g1[IO-all-DCM-0-8-5-00_k119_10027296:607-4321],class,Stramenopiles; Opalozoa; Bicoecea,Bicoecea,39.3
g1[IO-all-DCM-0-8-5-00_k119_10028232:1-797],class,Hacrobia; Cryptophyta; Cryptophyceae,Cryptophyceae,25.9
g1[IO-all-DCM-0-8-5-00_k119_10033933:4588-13506],order,Stramenopiles; Ochrophyta; Raphidophyceae; Rap...,Raphidophyceae_X,52.9
g1[IO-all-DCM-0-8-5-00_k119_10044618:374-4385],class,Stramenopiles; Ochrophyta; Pelagophyceae,Pelagophyceae,43.8
g1[IO-all-DCM-0-8-5-00_k119_10051159:95-6545],class,Archaeplastida; Chlorophyta; Chlorodendrophyceae,Chlorodendrophyceae,37.4
...,...,...,...,...
g4[IO-all-DCM-0-8-5-00_k119_16450953:4064-4594],order,Alveolata; Dinoflagellata; Dinophyceae; Gonyau...,Gonyaulacales,61.5
g4[IO-all-DCM-0-8-5-00_k119_2469638:7354-7692],order,Stramenopiles; Opalozoa; Bicoecea; Anoecales,Anoecales,54.5
g4[IO-all-DCM-0-8-5-00_k119_4804390:4241-5176],class,Stramenopiles; Ochrophyta; Bacillariophyta,Bacillariophyta,36.8
g4[IO-all-DCM-0-8-5-00_k119_7460344:12265-14439],class,Stramenopiles; Ochrophyta; Raphidophyceae,Raphidophyceae,47.9


In [128]:
levels = ['supergroup','division','class','order','family','genus','species']
tmp_df = pd.DataFrame(classification_df.full_classification.str.split('; '))
for cl in tmp_df.index:
    lineage = tmp_df.loc[cl,'full_classification']
    for i,l in enumerate(levels):
        if len(lineage)>i:
            outdf.loc[cl,l]=lineage[i]
        else:
            outdf.loc[cl,l]=np.nan
    

In [129]:
tax_dict = {}
total_len=len(outdf)
for l in levels:
    tax_dict[l]={}
    column_sum = outdf.groupby(l)[l].sum()
    column_count = outdf.groupby(l)[l].count()
    norm_count=column_count/total_len
    tax_dict[l]=norm_count



In [156]:
max_df = pd.DataFrame(index=levels, columns=['max_taxa','percent_id'])
for key in tax_dict:
    highest_tax = tax_dict[key][tax_dict[key]==tax_dict[key].max()].index
    max_val = tax_dict[key].max()
    max_df.loc[key]=highest_tax[0], max_val
    

In [174]:
outdf = pd.read_csv('output/MAGs/SAO-all-SRF-0-8-5-00_bin-59-estimated-taxonomy.out', sep='\t', index_col=0)

levels = ['supergroup','division','class','order','family','genus','species']
tmp_df = pd.DataFrame(outdf.full_classification.str.split('; '))
for cl in tmp_df.index:
    lineage = tmp_df.loc[cl,'full_classification']
    for i,l in enumerate(levels):
        if len(lineage)>i:
            outdf.loc[cl,l]=lineage[i]
        else:
            outdf.loc[cl,l]=np.nan

tax_dict = {}
total_len=len(outdf)
for l in levels:
    tax_dict[l]={}
    column_sum = outdf.groupby(l)[l].sum()
    column_count = outdf.groupby(l)[l].count()
    norm_count=column_count/total_len
    tax_dict[l]=norm_count

    
max_df = pd.DataFrame(index=levels, columns=['max_taxa','percent_id'])
for key in tax_dict:
    highest_tax = tax_dict[key][tax_dict[key]==tax_dict[key].max()].index
    max_val = tax_dict[key].max()
    max_df.loc[key]=highest_tax[0], max_val



In [175]:
max_df

,max_taxa,percent_id
supergroup,Stramenopiles,0.547887
division,Ochrophyta,0.457746
class,Bacillariophyta,0.11831
order,Raphidophyceae_X,0.0633803
family,Raphidophyceae_XX,0.0183099
genus,Florenciella,0.0028169
species,Coccolithus pelagicus ssp braarudi,0.00140845


In [98]:
for t,dd in df.groupby('qseqid'):
    dd

In [7]:
mfi = '/vortexfs1/omics/alexander/data/mmetsp/reference_dir/protein-species-map.json'
with open(mfi, 'rb') as fp:
    imp = ujson.load(fp)


In [33]:
tax_table

,cyverse_path,source_id_dup,ref_status,strain,supergroup,division,class,order,family,genus,species,notes
source_id,,,,,,,,,,,,
MMETSP0290,2225.0,MMETSP0290,Clean,CCMP2878,Alveolata,Apicomplexa,Colpodellidea,Colpodellida,Chromeraceae,Chromera,Chromera velia,Curated by J. del Campo (PR2)
MMETSP0288,2224.0,MMETSP0288,Clean,CCMP3155,Alveolata,Apicomplexa,Colpodellidea,Vitrelladida,Vitrellaceae,Vitrella,Vitrella brassicaformis,Adl et al. 2019; Curated by J. del Campo and V...
MMETSP1451,2282.0,MMETSP1451,Clean,CCMP3346,Alveolata,Apicomplexa,Colpodellidea,Vitrelladida,Vitrellaceae,Vitrella,Vitrella brassicaformis,Adl et al. 2019; Curated by J. del Campo and V...
MMETSP0372,1822.0,MMETSP0372,Clean,Grappler Inlet BC,Alveolata,Apicomplexa,Gregarinomorphea,Eugregarinorida,Lecudinidae,Lankesteria,Lankesteria abbottii,Adl et al. 2019; Curated by J. del Campo (PR2)
MMETSP0125,1740.0,MMETSP0125,Clean,ATCC 50986,Alveolata,Ciliophora,Colpodea,Colpodea_X,Cyrtolophosidida,Aristerostoma,Aristerostoma sp.,Ciliophora EukRef curation Boscaro V.; previou...
...,...,...,...,...,...,...,...,...,...,...,...,...
MMETSP0990,1938.0,MMETSP0990,Clean,CCMP2098,Stramenopiles,Ochrophyta,Dictyochophyceae,Dictyochophyceae_X,Dictyochophyceae_XX,Uncertain,Uncertain,Not described; listed as Dictyochophyceae
MMETSP0991,1943.0,MMETSP0991,Clean,CCMP2098,Stramenopiles,Ochrophyta,Dictyochophyceae,Dictyochophyceae_X,Dictyochophyceae_XX,Uncertain,Uncertain,Not described; listed as Dictyochophyceae
MMETSP0992,1944.0,MMETSP0992,Clean,CCMP2098,Stramenopiles,Ochrophyta,Dictyochophyceae,Dictyochophyceae_X,Dictyochophyceae_XX,Uncertain,Uncertain,Not described; listed as Dictyochophyceae
